# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [1]:
## The following libraries might be useful
!pip install -q langchain-openai
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q datasets
!pip install -U -q ragas
!pip install -U -q rouge_score

In [3]:
!pip install python-magic python-magic-bin

#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [ ]:
# Load the files as documents

In [5]:
from langchain_community.document_loaders import DirectoryLoader

# Replace 'your_folder_path' with the path to your folder
loader = DirectoryLoader(
    path="C:/Users/Anwita/Desktop/Upgrad/Course 5 - Generative AI (NLP)/RAG Assignment/rag_legal/corpus", 
    glob="**/*.txt"  # Loads .txt files recursively
)
documents = loader.load()

for doc in documents:
    print(f"File Path: {doc.metadata['source']}")
    print(f"Content: {doc.page_content[:200]}...")  # Print first 200 chars


File Path: C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\01_Bosch-Automotive-Service-Solutions-Mutual-Non-Disclosure-Agreement-7-12-17.txt
Content: MUTUAL NON-DISCLOSURE AGREEMENT

Between

AND

Subject Matter:

Effective Date of Agreement: Period , 2017

for Exchange of Information: , 2017 to

Period of Confidentiality:

THIS AGREEMENT is made a...
File Path: C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\12032018_NDA_The_20Munt_EN.txt
Content: Confidentiality Agreement

This Confidentiality Agreement (the “Agreement”) has been prepared on behalf of bpost SA/NV, a limited liability company of public law, and the City of Brussels (the “Seller...
File Path: C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\5-NSK-Confidentiality-Agreement-for-Suppliers.txt
Content: NON-DISCLOSURE AGREEMENT

This Agreement is effecti

In [44]:
import sys
print(sys.executable)

C:\Users\Anwita\anaconda3\python.exe


In [46]:
!pip show unstructured

Name: unstructured
Version: 0.18.13
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: C:\Users\Anwita\anaconda3\Lib\site-packages
Requires: backoff, beautifulsoup4, charset-normalizer, dataclasses-json, emoji, filetype, html5lib, langdetect, lxml, nltk, numpy, psutil, python-iso639, python-magic, python-oxmsg, rapidfuzz, requests, tqdm, typing-extensions, unstructured-client, wrapt
Required-by: 


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [7]:
# Clean and preprocess the data
import re
import logging
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords if not already done
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Remove email addresses
    text = re.sub(r'\S+@\S+', ' ', text)
    # Remove phone numbers (simple patterns)
    text = re.sub(r'\b\d{10}\b|\+?\d[\d -]{8,}\d', ' ', text)
    # Remove special characters except space and basic punctuation
    text = re.sub(r'[^\w\s,.]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    # Remove stopwords
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

# Logging setup
logging.basicConfig(filename='preprocessing.log', level=logging.ERROR)

preprocessed_docs = []
for doc in documents:
    try:
        clean_text = preprocess_text(doc.page_content)  # use .page_content for the actual text
        # Try the most likely keys for filename/source info
        file_path = doc.metadata.get('file_path', doc.metadata.get('source', 'unknown'))
        preprocessed_docs.append({
            'file_path': file_path,
            'content': clean_text
        })
    except Exception as e:
        file_path = doc.metadata.get('file_path', doc.metadata.get('source', 'unknown'))
        logging.error(f"Error processing {file_path}: {e}")
        continue

# preprocessed_docs now contains cleaned data for analysis

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anwita\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anwita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [26]:
# Calculate the average, maximum and minimum document length.

doc_lengths = [len(doc['content']) for doc in preprocessed_docs]

average_length = sum(doc_lengths) / len(doc_lengths)
max_length = max(doc_lengths)
min_length = min(doc_lengths)

print(f"Average document length: {average_length:.2f}")
print(f"Maximum document length: {max_length}")
print(f"Minimum document length: {min_length}")

Average document length: 136761.13
Maximum document length: 719909
Minimum document length: 1381


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [9]:
# Find frequency of occurence of words

from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Combine all cleaned document contents
all_text = ' '.join([doc['content'] for doc in preprocessed_docs])

# Tokenize and remove stop words again (for extra safety)
stop_words = set(stopwords.words('english'))
words = word_tokenize(all_text)
words_filtered = [word for word in words if word.isalpha() and word not in stop_words]

# Count frequency of each word
freq_counter = Counter(words_filtered)

# 20 most common words
most_common = freq_counter.most_common(20)
print("20 Most Common Words:")
for word, freq in most_common:
    print(f"{word}: {freq}")

# 20 least common words
least_common = freq_counter.most_common()[:-21:-1]
print("\n20 Least Common Words:")
for word, freq in least_common:
    print(f"{word}: {freq}")


20 Most Common Words:
company: 144438
agreement: 64313
section: 62936
shall: 62561
parent: 60327
merger: 33533
subsidiaries: 32751
date: 30075
material: 29998
time: 26975
stock: 25340
b: 24787
applicable: 24404
party: 23211
respect: 22839
including: 20124
shares: 19345
ii: 18618
prior: 18173
effect: 17692

20 Least Common Words:
collins: 1
turned: 1
newer: 1
continual: 1
strive: 1
configuring: 1
vungle: 1
tapresearch: 1
faq: 1
soomla: 1
smaato: 1
pinsight: 1
nativex: 1
motive: 1
loopme: 1
intl: 1
lifestreet: 1
kongregate: 1
inmobi: 1
hyprmx: 1


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

In [13]:
# Transform the page contents of documents
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming your cleaned document texts are present in preprocessed_docs with the 'content' key
documents_text = [doc['content'] for doc in preprocessed_docs]

# Create the TF-IDF matrix
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(documents_text)

In [15]:
# Compute similarity scores
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculate the cosine similarity matrix for all documents
similarity_matrix = cosine_similarity(tfidf_matrix)

# For first 10 documents:
similarity_first_10 = similarity_matrix[:10, :10]
print("Similarity matrix for the first 10 documents:")
print(np.round(similarity_first_10, 2))

Similarity matrix for the first 10 documents:
[[1.   0.3  0.7  0.62 0.62 0.68 0.69 0.74 0.6  0.7 ]
 [0.3  1.   0.29 0.23 0.25 0.26 0.25 0.27 0.32 0.3 ]
 [0.7  0.29 1.   0.77 0.79 0.8  0.81 0.83 0.54 0.82]
 [0.62 0.23 0.77 1.   0.7  0.74 0.76 0.74 0.42 0.73]
 [0.62 0.25 0.79 0.7  1.   0.7  0.71 0.72 0.43 0.74]
 [0.68 0.26 0.8  0.74 0.7  1.   0.82 0.72 0.5  0.81]
 [0.69 0.25 0.81 0.76 0.71 0.82 1.   0.76 0.51 0.81]
 [0.74 0.27 0.83 0.74 0.72 0.72 0.76 1.   0.46 0.75]
 [0.6  0.32 0.54 0.42 0.43 0.5  0.51 0.46 1.   0.57]
 [0.7  0.3  0.82 0.73 0.74 0.81 0.81 0.75 0.57 1.  ]]


In [17]:
# create a list of 10 random integers
import random

n_docs = tfidf_matrix.shape[0]  # This is the correct way!
random_indices = random.sample(range(n_docs), 10)
print("Randomly selected document indices:", random_indices)

Randomly selected document indices: [24, 246, 168, 221, 205, 5, 148, 80, 213, 103]


In [19]:
# Compute similarity scores for 10 random documents
# Select the TF-IDF vectors for the chosen random documents
tfidf_random_10 = tfidf_matrix[random_indices]
similarity_random_10 = cosine_similarity(tfidf_random_10)

print("Similarity matrix for 10 random documents:")
print(np.round(similarity_random_10, 2))

Similarity matrix for 10 random documents:
[[1.   0.13 0.22 0.21 0.21 0.75 0.24 0.58 0.18 0.18]
 [0.13 1.   0.31 0.31 0.31 0.11 0.33 0.13 0.28 0.09]
 [0.22 0.31 1.   0.87 0.84 0.18 0.86 0.23 0.79 0.15]
 [0.21 0.31 0.87 1.   0.91 0.17 0.81 0.21 0.87 0.14]
 [0.21 0.31 0.84 0.91 1.   0.16 0.8  0.21 0.91 0.13]
 [0.75 0.11 0.18 0.17 0.16 1.   0.21 0.66 0.14 0.18]
 [0.24 0.33 0.86 0.81 0.8  0.21 1.   0.26 0.79 0.16]
 [0.58 0.13 0.23 0.21 0.21 0.66 0.26 1.   0.19 0.17]
 [0.18 0.28 0.79 0.87 0.91 0.14 0.79 0.19 1.   0.12]
 [0.18 0.09 0.15 0.14 0.13 0.18 0.16 0.17 0.12 1.  ]]


### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [21]:
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """
    Split a string into overlapping chunks of specified size, with optional overlap.
    Returns a generator yielding each chunk.
    """
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + chunk_size, text_len)
        yield text[start:end]
        start = end - chunk_overlap if (end - chunk_overlap) > start else end

# Save chunks to disk or process them one at a time to avoid keeping all in RAM.
# Here, we just print out one example chunk per document, avoiding a big list.

for idx, doc in enumerate(preprocessed_docs):
    print(f"Document {idx} ({doc.get('file_path','unknown')}):")
    for i, chunk in enumerate(chunk_text(doc['content'], chunk_size=500, chunk_overlap=50)):
        # Example: process chunk here (e.g., save to disk, add to DB, or work with it)
        if i == 0:  # Only print the FIRST chunk per document to demonstrate
            print(f"  Chunk 0: {chunk[:100]}...")  # Show first 100 characters
        # Don't accumulate all chunks in a list

# If you need all chunks to be available later, consider writing each chunk individually to disk,
# a database, or only hold a small batch in memory at a time.

Document 0 (C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\01_Bosch-Automotive-Service-Solutions-Mutual-Non-Disclosure-Agreement-7-12-17.txt):
  Chunk 0: mutual non disclosure agreement subject matter effective date agreement period , 2017 exchange infor...
Document 1 (C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\12032018_NDA_The_20Munt_EN.txt):
  Chunk 0: confidentiality agreement confidentiality agreement agreement prepared behalf bpost sa nv , limited ...
Document 2 (C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\5-NSK-Confidentiality-Agreement-for-Suppliers.txt):
  Chunk 0: non disclosure agreement agreement effective day 2019 effective date 1 nsk europe limited , register...
Document 3 (C:\Users\Anwita\Desktop\Upgrad\Course 5 - Generative AI (NLP)\RAG Assignment\rag_legal\corpus\contractnli\ADVANIDE-NON-D

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [23]:
# Fetch your OPENAI API Key as an environment variable
import os

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    print("OPENAI_API_KEY environment variable not found. Please set it before running embedding functions.")
else:
    print("API Key loaded successfully.")

API Key loaded successfully.


In [25]:
# Initialise an embedding function
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

def embed_texts(texts):
    return embedding_function.embed_documents(texts)

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [27]:
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """
    Split a string into overlapping chunks of specified size, with optional overlap.
    Returns a generator yielding each chunk.
    """
    start = 0
    textlen = len(text)
    while start < textlen:
        end = min(start + chunk_size, textlen)
        yield text[start:end]
        # Step forward by chunk_size - chunk_overlap, but don't step backward.
        start = end - chunk_overlap if end - chunk_overlap > start else end

In [ ]:
from langchain_community.vectorstores import Chroma
import os

# ------ Step 1: Gather all chunks with metadata --------
chunks_for_db = []
chunk_size = 500
chunk_overlap = 50

# Use your existing chunk_text function and preprocessed_docs
for doc_idx, doc in enumerate(preprocessed_docs):
    file_path = doc.get('file_path', 'unknown')
    chunks = list(chunk_text(doc['content'], chunk_size=chunk_size, chunk_overlap=chunk_overlap))
    for i, chunk in enumerate(chunks):
        chunks_for_db.append({
            'content': chunk,
            'metadata': {
                'file_path': file_path,
                'chunk_id': i
            }
        })

# ------ Step 2: Prepare texts and metadata for embedding -------
docs_to_embed = [item['content'] for item in chunks_for_db]
metadatas = [item['metadata'] for item in chunks_for_db]

# ------ Step 3: Create embedding function -------
# Make sure you've imported and initialized your embedding function earlier, e.g.:
# from langchain_openai import OpenAIEmbeddings
# embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

# ------ Step 4: Create directory for vector DB ------
db_dir = "vector_db_chroma"
os.makedirs(db_dir, exist_ok=True)

# ------ Step 5: Create and persist the Chroma vector DB using from_texts -------
vector_db = Chroma.from_texts(
    texts=docs_to_embed,
    embedding=embedding_function,
    metadatas=metadatas,
    persist_directory=db_dir
)

vector_db.persist()  # Save DB to disk

print(f"Chroma vector DB created and saved to {db_dir}")


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [ ]:
# Create a RAG chain

In [15]:
!pip install -q langchain-openai langchain-community langchain-chroma

In [5]:
# --- Section 2.2.1: Create a RAG Chain ---

# Step 1: Import required modules (these have already been introduced earlier)
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Step 2: Load your persisted Chroma vector database from disk
# (db_dir must be set as the same directory used in your previous Chroma code cell)
vector_db = Chroma(
    embedding_function=embedding_function,  # already defined when creating the DB
    persist_directory=db_dir  # e.g., "vector_db_chroma"
)

# Step 3: Initialize LLM (already imported in notebook)
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # You may use "gpt-4" if available in your API
    openai_api_key=openai_api_key,  # Already set previously
    temperature=0  # Deterministic output for reproducibility
)

# Step 4: Convert Chroma DB to retriever
retriever = vector_db.as_retriever()

# Step 5: Build the RAG Chain using RetrievalQA from LangChain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Standard QA type
    retriever=retriever,
    return_source_documents=True  # Also provide retrieved source docs with answer
)

# RAG chain is now ready to use:
result = rag_chain({"query": "What is the duration of confidentiality period in the NDA agreements?"})
print(result["result"])
print(result["source_documents"])


NameError: name 'db_dir' is not defined

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [ ]:
# Create a function for question answering
import json
import glob

question_set = []
answer_set = []
for file in glob.glob("path/to/benchmarks/*.json"):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        for test in data.get("tests", []):
            question_set.append(test.get("query"))
            for snippet in test.get("snippets", []):
                answer = snippet.get("answer")
                if answer:
                    answer_set.append(answer)


In [ ]:
# Example question
# question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"


## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [ ]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set



#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [ ]:
# Function to evaluate the RAG pipeline
from rouge_score import rouge_scorer
from ragas.metrics import ragas_metric  # Example, adapt for actual usage

def evaluate_answer(predicted, ground_truth):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(predicted, ground_truth)
    # For Bleu, you can use nltk or sacrebleu
    # For Ragas, refer to the actual ragas library for implementation
    return scores

# Evaluate on first 100 pairs
for i in range(100):
    pred = rag_chain({"query": question_set[i]})["result"]
    gt = answer_set[i]
    metrics = evaluate_answer(pred, gt)
    print(f"Q{i}: {metrics}")

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [ ]:
# Evaluate the RAG pipeline


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.